# Compares original data and generated parameters to spot issues

In [223]:
import pandas as pd
import numpy as np
from datetime import datetime
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
from timezonefinder import TimezoneFinder
import astropy.units as u
ZoneFinder = TimezoneFinder()

In [224]:
def get_time_zone(latitude,longitude):
    #Returns an astropy timedelta object with correct UTC offset
    zone_name = ZoneFinder.timezone_at(lat=latitude,lng=longitude)
    utc_offset = datetime.now(ZoneInfo(zone_name)).utcoffset().total_seconds()
    return TimeDelta(utc_offset*u.second)

def get_local_times(obs_dates, latitude,longitude):
    local_times = np.empty(len(obs_dates),dtype=datetime)
    obs_dates  =Time(obs_dates)
    for i, date in enumerate(obs_dates):
          timezone = get_time_zone(latitude[i],longitude[i])     
          local_obs_time = date + timezone
          obs_time = local_obs_time.to_datetime().time()
          local_times[i] = obs_time
    return local_times
  

In [225]:
icouk_data_file = '..\\Data\\icouk_sighting_data_with_params.csv'
icop_data_file = '..\\Data\\icop_ahmed_2020_sighting_data_with_params.csv'
alrefay_data_file = '..\\Data\\alrefay_2018_sighting_data_with_params.csv'
#allawi_data_file = '..\\Data\\schaefer_odeh_allawi_2022_sighting_data_with_params.csv'
yallop_data_file = '..\\Data\\yallop_sighting_data_with_params.csv'

icouk_data_file_orig = '..\\Data\\icouk_sighting_data.csv'
icop_data_file_orig = '..\\Data\\icop_ahmed_2020_sighting_data.csv'
alrefay_data_file_orig = '..\\Data\\alrefay_2018_sighting_data.csv'
#allawi_data_file_orig = '..\\Data\\schaefer_odeh_allawi_2022_sighting_data.csv'
yallop_data_file_orig = '..\\Data\\yallop_sighting_data.csv'

icouk_data = pd.read_csv(icouk_data_file)
icop_data = pd.read_csv(icop_data_file)
alrefay_data = pd.read_csv(alrefay_data_file)
yallop_data = pd.read_csv(yallop_data_file)

icouk_data_orig = pd.read_csv(icouk_data_file_orig)
icop_data_orig = pd.read_csv(icop_data_file_orig)
alrefay_data_orig = pd.read_csv(alrefay_data_file_orig)
yallop_data_orig = pd.read_csv(yallop_data_file_orig)

In [226]:
#ICOUK file

num_of_rows = icouk_data.shape[0]
cols=["Orig Date", "Date", "Date diff","Local Time","Local Sunset", "Local Moonset", "Lat", "Lon", "q", "Seen"]
data = pd.DataFrame(index=np.arange(0, num_of_rows), columns=cols[0:-5])
data.index.name="Index"

orig_dates = Time.strptime(icouk_data_orig["Date"].to_numpy().astype(str), "%d-%b-%y").to_datetime()
dates = Time(icouk_data["Date"],format="jd").to_datetime()

data["Orig Date"] = Time(orig_dates).strftime("%d/%m/%Y")
data["Date"] = Time(dates).strftime("%d/%m/%Y %H:%M")
data["Date diff"] = dates - orig_dates
data["Local Time"]  = get_local_times(dates ,icouk_data["Latitude"],icouk_data["Longitude"] )
data["Local Sunset"]  = get_local_times(Time(icouk_data["Sunset"],format="jd"),icouk_data["Latitude"],icouk_data["Longitude"] )
data["Local Moonset"]  = get_local_times(Time(icouk_data["Moonset"],format="jd"),icouk_data["Latitude"],icouk_data["Longitude"] )
data["Lat"] = icouk_data["Latitude"]
data["Lon"] = icouk_data["Longitude"]
data["q"] = icouk_data["q'"]
data["Seen"] = icouk_data_orig["Seen?"]

data.to_csv("..\\Data\\Validation\\icouk_validation.csv")


In [227]:
#ICOP file

num_of_rows = icop_data.shape[0]
cols=["Orig Date", "Date", "Date diff", "Local Time","Local Sunset", "Local Moonset","Lat", "Lon", "Orig Sunset", "Sunset", "Sunset Diff", "q", "Seen"]
data = pd.DataFrame(index=np.arange(0, num_of_rows), columns=cols[0:-5])
data.index.name="Index"

orig_dates = Time.strptime(icop_data_orig["Date"].to_numpy().astype(str), "%d/%m/%Y").to_datetime()
dates = Time(icop_data["Date"],format="jd").to_datetime()

data["Orig Date"] = Time(orig_dates).strftime("%d/%m/%Y")
data["Date"] = Time(dates).strftime("%d/%m/%Y %H:%M")
data["Date diff"] = dates - orig_dates

data["Local Time"]  = get_local_times(dates ,icop_data["Latitude"],icop_data["Longitude"] )
data["Local Sunset"]  = get_local_times(Time(icop_data["Sunset"],format="jd"),icop_data["Latitude"],icop_data["Longitude"] )
data["Local Moonset"]  = get_local_times(Time(icop_data["Moonset"],format="jd"),icop_data["Latitude"],icop_data["Longitude"] )

orig_sunsets = Time.strptime(icop_data_orig["sunset"].to_numpy().astype(str), "%d/%m/%Y %H:%M").to_datetime()
sunsets = Time(icop_data["Sunset"],format="jd").to_datetime()

data["Orig Sunset"] = orig_sunsets
data["Sunset"] = sunsets
data["Sunset diff"] = sunsets - orig_sunsets

data["Lat"] = icop_data["Latitude"]
data["Lon"] = icop_data["Longitude"]
data["q"] = icop_data["q'"]
data["Seen"] = icop_data["Seen"]

data.to_csv("..\\Data\\Validation\\icop_validation.csv")

In [228]:
#Alrefay file

num_of_rows = alrefay_data.shape[0]
cols=["Orig Date", "Date", "Date diff", "Local Time","Local Sunset", "Local Moonset","Lat", "Lon", "Orig ARCV", "ARCV", "ARCV Diff", "q", "Seen"]
data = pd.DataFrame(index=np.arange(0, num_of_rows), columns=cols[0:-5])
data.index.name="Index"

orig_dates = Time.strptime(alrefay_data_orig["Date"].to_numpy().astype(str), " %Y/%m/%d").to_datetime()
dates = Time(alrefay_data["Date"],format="jd").to_datetime()

data["Orig Date"] = Time(orig_dates).strftime("%d/%m/%Y")
data["Date"] = Time(dates).strftime("%d/%m/%Y %H:%M")
data["Date diff"] = dates - orig_dates

data["Local Time"]  = get_local_times(dates ,alrefay_data["Latitude"],alrefay_data["Longitude"] )
data["Local Sunset"]  = get_local_times(Time(alrefay_data["Sunset"],format="jd"),alrefay_data["Latitude"],alrefay_data["Longitude"] )
data["Local Moonset"]  = get_local_times(Time(alrefay_data["Moonset"],format="jd"),alrefay_data["Latitude"],alrefay_data["Longitude"] )

data["Orig ARCV"] = alrefay_data_orig[" ARCV"]
data["ARCV"] = alrefay_data["ARCV"]
data["ARCV diff"] = round(alrefay_data["ARCV"] - alrefay_data_orig[" ARCV"],5)

data["Lat"] = alrefay_data["Latitude"]
data["Lon"] = alrefay_data["Longitude"]
data["q"] = alrefay_data["q'"]
data["Seen"] = alrefay_data["Seen"]

data.to_csv("..\\Data\\Validation\\alrefay_validation.csv")

In [229]:
#Yallop file

num_of_rows = yallop_data.shape[0]
cols=["Orig Date", "Date", "Date diff", "Local Time", "Lat", "Lon", "Local Sunset", "Local Moonset", "Orig ARCV", "ARCV", "ARCV Diff", "Orig q", "q", "q Diff", "Seen"]
data = pd.DataFrame(index=np.arange(0, num_of_rows), columns=cols[0:-5])
data.index.name="Index"

date_text = yallop_data_orig['D'].astype(str)+"/"+yallop_data_orig['M'].astype(str)+"/"+yallop_data_orig['Y'].astype(str)
orig_dates = Time.strptime(date_text.to_numpy().astype(str), "%d/%m/%Y").to_datetime()

dates = Time(yallop_data["Date"],format="jd").to_datetime()

data["Orig Date"] = Time(orig_dates).strftime("%d/%m/%Y")
data["Date"] = Time(dates).strftime("%d/%m/%Y %H:%M")
data["Date diff"] = dates - orig_dates

data["Local Time"]  = get_local_times(dates,yallop_data["Latitude"],yallop_data["Longitude"] )
data["Local Sunset"]  = get_local_times(Time(yallop_data["Sunset"],format="jd"),yallop_data["Latitude"],yallop_data["Longitude"] )
data["Local Moonset"]  = get_local_times(Time(yallop_data["Moonset"],format="jd"),yallop_data["Latitude"],yallop_data["Longitude"] )

data["Orig ARCV"] = yallop_data_orig["ARCV"]
data["ARCV"] = yallop_data["ARCV"]
data["ARCV diff"] = round(yallop_data["ARCV"] - yallop_data_orig["ARCV"],5)

data["Orig q"] = yallop_data_orig["q"]
data["q"] = yallop_data["q'"]
data["q diff"] = round(yallop_data["q'"] - yallop_data_orig["q"],5)

data["Lat"] = yallop_data["Latitude"]
data["Lon"] = yallop_data["Longitude"]
data["q"] = yallop_data["q'"]
data["Seen"] = yallop_data["Seen"]
data["M/E"] = yallop_data_orig["M/E"]

data.to_csv("..\\Data\\Validation\\yallop_validation.csv")

c:\ProgramData\miniconda3\envs\mphys-moon\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 125 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
c:\ProgramData\miniconda3\envs\mphys-moon\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 125 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
c:\ProgramData\miniconda3\envs\mphys-moon\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
c:\ProgramData\miniconda3\envs\mphys-moon\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
c:\ProgramData\miniconda3\envs\mphys-moon\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA functi